In [194]:
# %%
import os,sys,inspect
currentdir = "D:\BAMT\plan\\big_cluster"
parentdir = os.path.dirname(currentdir)
parentparentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentparentdir)

# %%
import pandas as pd
import random
import numpy as np
from bayesian.save_bn import save_structure, save_params, read_structure, read_params

from copy import copy


import matplotlib.pyplot as plt
from core.core_utils import project_root


from sklearn.metrics import accuracy_score, mean_squared_error

from graph.precision_recall import child_dict, precision_recall
import json5


In [195]:
name = 'diabetes'
values = [1500, 3000]

In [196]:
ind = 0
time_list = []
with open(f'{currentdir}/log_full_{name}.txt', 'r') as log:
    for i, line in enumerate(log):
        if i % 3 == 0:
            large = line.rstrip()
        elif i % 3 == 1:
            col_new = json5.loads(' '.join(line.split(' ')[1:]).rstrip())
        else:
            time_long = float(line.split(' ')[-1].rstrip())
            time_list.append(time_long)
            

In [197]:
time_list

[92306.81080079079, 92313.60185647011]

In [198]:
coded_dict = dict()   
prev_alpha = 1.0
ind = -1
with open(f'{currentdir}/log_coded_{name}.txt', 'r') as log:
    for i, line in enumerate(log):
        if i % 5 == 1:
            alpha = float(line.rstrip())
        elif i % 5 == 2:
            code_type = line.rstrip()
            if (alpha <= prev_alpha) and (code_type == 'hamming'):
            #if (alpha <= prev_alpha):
                ind += 1
                samples = values[ind % len(values)]
            prev_alpha = alpha
        else:
            if i % 5 == 4:
                time_long = float(line.split(' ')[-1].rstrip())
                coded_dict[str({'name': name, 'alpha': alpha, 'code_type': code_type, 'samples': samples})] = time_long
                

In [199]:
coded_dict

{"{'name': 'diabetes', 'alpha': 0.2, 'code_type': 'mostly', 'samples': 1500}": 89939.65447092056,
 "{'name': 'diabetes', 'alpha': 0.3, 'code_type': 'mostly', 'samples': 1500}": 62053.39473319054,
 "{'name': 'diabetes', 'alpha': 0.4, 'code_type': 'mostly', 'samples': 1500}": 36113.97789692879,
 "{'name': 'diabetes', 'alpha': 0.5, 'code_type': 'mostly', 'samples': 1500}": 25941.601482868195,
 "{'name': 'diabetes', 'alpha': 0.6, 'code_type': 'mostly', 'samples': 1500}": 16260.80009150505,
 "{'name': 'diabetes', 'alpha': 0.2, 'code_type': 'hamming', 'samples': 1500}": 90072.81922006607,
 "{'name': 'diabetes', 'alpha': 0.3, 'code_type': 'hamming', 'samples': 1500}": 58485.07824230194,
 "{'name': 'diabetes', 'alpha': 0.4, 'code_type': 'hamming', 'samples': 1500}": 32475.894589185715,
 "{'name': 'diabetes', 'alpha': 0.5, 'code_type': 'hamming', 'samples': 1500}": 23664.275516033173,
 "{'name': 'diabetes', 'alpha': 0.6, 'code_type': 'hamming', 'samples': 1500}": 15609.107548475266,
 "{'name': 

In [200]:
result = pd.DataFrame(columns=['name', 'alpha', 'code_type', 'samples', 'AP', 'AR', 'AHP', 'AHR', 'SHD', 'true_edges_len', 'time'])

In [201]:
for root, dirs, files in os.walk(os.path.join(parentparentdir, "models", "structure_bn")):
    for file in files:
        if name in file:
            att = file.split('_')
            bn = read_structure(file.replace('.txt', ''))
            corr_edges = [e for e in bn.E if not any([')_in' in node or ')_out' in node for node in e])]
            true_edges = read_structure(f'{name}_true').E
            true_edges_len = len(read_structure(f'{name}_true').E)
            net_stat = precision_recall(corr_edges, true_edges)
            if att[0] == 'coded':
                cur_time = -1.0
                alpha = float('0.' + att[2])
                code_type = att[3]
                samples = int(att[4].replace('.txt', ''))
                id = {'name': name, 'alpha': alpha, 'code_type': code_type, 'samples': samples}
                
                for key, val in coded_dict.items():
                    dict_key = json5.loads(key)
                    if id == dict_key:
                        cur_time = val
                if cur_time >= 0:
                    result = result.append({**id, **net_stat, 'true_edges_len': true_edges_len, 'time': cur_time}, ignore_index=True)
            if att[0] == 'uncoded':
                for i, value in enumerate(values):
                    result = result.append({'name': name, 'alpha': 0.0, 'code_type': 'none', 'samples': value, **net_stat, 'true_edges_len': true_edges_len, 'time': time_list[i]}, ignore_index=True)
            

In [202]:
result.head()

,name,alpha,code_type,samples,AP,AR,AHP,AHR,SHD,true_edges_len,time
0,diabetes,0.2,hamming,1500,0.01,0.01,0.01,0.01,1099,602,90072.819220
1,diabetes,0.2,mostly,1500,0.01,0.01,0.01,0.01,1096,602,89939.654471
2,diabetes,0.3,hamming,1500,0.12,0.12,0.09,0.08,1059,602,58485.078242
3,diabetes,0.3,mostly,1500,0.90,0.12,0.63,0.08,561,602,62053.394733
4,diabetes,0.4,hamming,1500,0.22,0.27,0.11,0.13,1095,602,32475.894589


In [203]:
result.to_csv(f'{name}_stat.csv', index=False)

In [204]:
# true_net = "[[\"alcoholism\", \"THepatitis\"], [\"alcoholism\", \"Steatosis\"], [\"vh_amn\", \"ChHepatitis\"], [\"vh_amn\", \"hbsag\"], [\"vh_amn\", \"hbsag_anti\"], [\"vh_amn\", \"hbc_anti\"], [\"vh_amn\", \"hcv_anti\"], [\"vh_amn\", \"hbeag\"], [\"hepatotoxic\", \"THepatitis\"], [\"hepatotoxic\", \"RHepatitis\"], [\"THepatitis\", \"fatigue\"], [\"THepatitis\", \"phosphatase\"], [\"THepatitis\", \"inr\"], [\"THepatitis\", \"hepatomegaly\"], [\"THepatitis\", \"alt\"], [\"THepatitis\", \"ast\"], [\"THepatitis\", \"ggtp\"], [\"THepatitis\", \"anorexia\"], [\"THepatitis\", \"nausea\"], [\"THepatitis\", \"spleen\"], [\"hospital\", \"injections\"], [\"hospital\", \"transfusion\"], [\"surgery\", \"injections\"], [\"surgery\", \"transfusion\"], [\"gallstones\", \"choledocholithotomy\"], [\"gallstones\", \"bilirubin\"], [\"gallstones\", \"upper_pain\"], [\"gallstones\", \"fat\"], [\"gallstones\", \"pressure_ruq\"], [\"gallstones\", \"flatulence\"], [\"gallstones\", \"amylase\"], [\"choledocholithotomy\", \"injections\"], [\"choledocholithotomy\", \"transfusion\"], [\"injections\", \"ChHepatitis\"], [\"transfusion\", \"ChHepatitis\"], [\"ChHepatitis\", \"fibrosis\"], [\"ChHepatitis\", \"fatigue\"], [\"ChHepatitis\", \"bilirubin\"], [\"ChHepatitis\", \"pressure_ruq\"], [\"ChHepatitis\", \"phosphatase\"], [\"ChHepatitis\", \"inr\"], [\"ChHepatitis\", \"ESR\"], [\"ChHepatitis\", \"alt\"], [\"ChHepatitis\", \"ast\"], [\"ChHepatitis\", \"ggtp\"], [\"ChHepatitis\", \"cholesterol\"], [\"ChHepatitis\", \"hbsag\"], [\"ChHepatitis\", \"hbsag_anti\"], [\"ChHepatitis\", \"hbc_anti\"], [\"ChHepatitis\", \"hcv_anti\"], [\"ChHepatitis\", \"hbeag\"], [\"sex\", \"PBC\"], [\"sex\", \"Hyperbilirubinemia\"], [\"age\", \"PBC\"], [\"age\", \"Hyperbilirubinemia\"], [\"PBC\", \"bilirubin\"], [\"PBC\", \"pressure_ruq\"], [\"PBC\", \"ama\"], [\"PBC\", \"le_cells\"], [\"PBC\", \"joints\"], [\"PBC\", \"pain\"], [\"PBC\", \"platelet\"], [\"PBC\", \"encephalopathy\"], [\"PBC\", \"ESR\"], [\"PBC\", \"ggtp\"], [\"PBC\", \"cholesterol\"], [\"PBC\", \"carcinoma\"], [\"fibrosis\", \"Cirrhosis\"], [\"diabetes\", \"obesity\"], [\"obesity\", \"Steatosis\"], [\"Steatosis\", \"Cirrhosis\"], [\"Steatosis\", \"triglycerides\"], [\"Steatosis\", \"pain_ruq\"], [\"Steatosis\", \"hepatomegaly\"], [\"Steatosis\", \"ESR\"], [\"Steatosis\", \"alt\"], [\"Steatosis\", \"ast\"], [\"Steatosis\", \"ggtp\"], [\"Steatosis\", \"cholesterol\"], [\"Cirrhosis\", \"bilirubin\"], [\"Cirrhosis\", \"phosphatase\"], [\"Cirrhosis\", \"proteins\"], [\"Cirrhosis\", \"edema\"], [\"Cirrhosis\", \"platelet\"], [\"Cirrhosis\", \"inr\"], [\"Cirrhosis\", \"alcohol\"], [\"Cirrhosis\", \"encephalopathy\"], [\"Cirrhosis\", \"alt\"], [\"Cirrhosis\", \"ast\"], [\"Cirrhosis\", \"spleen\"], [\"Cirrhosis\", \"spiders\"], [\"Cirrhosis\", \"albumin\"], [\"Cirrhosis\", \"edge\"], [\"Cirrhosis\", \"irregular_liver\"], [\"Cirrhosis\", \"palms\"], [\"Cirrhosis\", \"carcinoma\"], [\"Hyperbilirubinemia\", \"bilirubin\"], [\"Hyperbilirubinemia\", \"pain_ruq\"], [\"Hyperbilirubinemia\", \"inr\"], [\"Hyperbilirubinemia\", \"hepatomegaly\"], [\"Hyperbilirubinemia\", \"ESR\"], [\"Hyperbilirubinemia\", \"ggtp\"], [\"RHepatitis\", \"fatigue\"], [\"RHepatitis\", \"phosphatase\"], [\"RHepatitis\", \"hepatomegaly\"], [\"RHepatitis\", \"alt\"], [\"RHepatitis\", \"ast\"], [\"RHepatitis\", \"ggtp\"], [\"RHepatitis\", \"anorexia\"], [\"RHepatitis\", \"nausea\"], [\"RHepatitis\", \"spleen\"], [\"bilirubin\", \"itching\"], [\"bilirubin\", \"skin\"], [\"bilirubin\", \"jaundice\"], [\"joints\", \"pain\"], [\"proteins\", \"ascites\"], [\"platelet\", \"bleeding\"], [\"inr\", \"bleeding\"], [\"encephalopathy\", \"urea\"], [\"encephalopathy\", \"density\"], [\"encephalopathy\", \"consciousness\"], [\"hepatomegaly\", \"hepatalgia\"], [\"hbsag\", \"hbsag_anti\"]]"
# true_net = json5.loads(true_net)
# print(true_net)

In [205]:
# nodes = []
# for e in true_net:
#     nodes.extend(e)
# nodes = list(set(nodes))

In [206]:
# save_structure({'V': nodes, 'E': true_net}, 'hepar2_true')